## Weaviate workshop

<a target="_blank" href="https://colab.research.google.com/github/weaviate-tutorials/intro-workshop/blob/main/workshop.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Goals:

#### What you will see:


- Create a vector database with Weaviate,
- Add data to the database, and
- Interact with the data, including searching, and using LLMs with your data in Weaviate

### You will learn today:

- What Weaviate is,
- How it stores the data (based on its "meaning"), and
- What you can do with Weaviate, like semantic searches, and using LLMs to transform data.

Install the Weaviate python client, for environments that don't yet have it.

In [1]:
# !pip install -U --pre weaviate-client

## Preparation: Get the data

We'll use a subset of the Jeopardy! quiz library:
> https://www.kaggle.com/datasets/tunguz/200000-jeopardy-questions

Pre-processed version:
> https://raw.githubusercontent.com/databyjp/wv_demo_uploader/main/weaviate_datasets/data/jeopardy_1k.json


Load (or download) the data, and preview it

In [2]:
import requests
import json

# Download the data
response = requests.get('https://raw.githubusercontent.com/databyjp/wv_demo_uploader/main/weaviate_datasets/data/jeopardy_1k.json')
raw_data = response.text

# Parse the JSON and preview it
data = json.loads(raw_data)
print(type(data), len(data))
print(json.dumps(data[0], indent=2))

<class 'list'> 1000
{
  "Air Date": "2006-11-08",
  "Round": "Double Jeopardy!",
  "Value": 800,
  "Category": "AMERICAN HISTORY",
  "Question": "Abraham Lincoln died across the street from this theatre on April 15, 1865",
  "Answer": "Ford's Theatre (the Ford Theatre accepted)"
}


## Step 1: Create a Weaviate instance (database)

This (Embedded Weaviate) is a quick way to create a Weaviate database. Note that this is suitable for evaluation use only, and currently not compatible with Windows (we are working on it 😉).

You can also use:
- A free sandbox with Weaviate Cloud Services
- Open-source Weaviate directly, available cross-platform with Docker

In [3]:
import weaviate
import weaviate.classes as wvc
import os

client = weaviate.connect_to_embedded(
    version="1.23.0",
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")  # Replace this with your actual key
    }
)


Started /Users/jphwang/.cache/weaviate-embedded: process ID 93747


{"action":"startup","default_vectorizer_module":"none","level":"info","msg":"the default vectorizer modules is set to \"none\", as a result all new schema classes without an explicit vectorizer setting, will use this vectorizer","time":"2024-01-10T13:39:39Z"}
{"action":"startup","auto_schema_enabled":true,"level":"info","msg":"auto schema enabled setting is set to \"true\"","time":"2024-01-10T13:39:39Z"}
{"level":"info","msg":"No resource limits set, weaviate will use all available memory and CPU. To limit resources, set LIMIT_RESOURCES=true","time":"2024-01-10T13:39:39Z"}
{"level":"warning","msg":"Multiple vector spaces are present, GraphQL Explore and REST API list objects endpoint module include params has been disabled as a result.","time":"2024-01-10T13:39:39Z"}
{"action":"grpc_startup","level":"info","msg":"grpc server listening at [::]:50051","time":"2024-01-10T13:39:39Z"}
{"action":"restapi_management","level":"info","msg":"Serving weaviate at http://127.0.0.1:8079","time":"202

Retrieve Weaviate instance information to check our configuration.

In [4]:
client.get_meta()

{'hostname': 'http://127.0.0.1:8079',
 'modules': {'generative-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'Generative Search - OpenAI'},
  'qna-openai': {'documentationHref': 'https://platform.openai.com/docs/api-reference/completions',
   'name': 'OpenAI Question & Answering Module'},
  'ref2vec-centroid': {},
  'reranker-cohere': {'documentationHref': 'https://txt.cohere.com/rerank/',
   'name': 'Reranker - Cohere'},
  'text2vec-cohere': {'documentationHref': 'https://docs.cohere.ai/embedding-wiki/',
   'name': 'Cohere Module'},
  'text2vec-huggingface': {'documentationHref': 'https://huggingface.co/docs/api-inference/detailed_parameters#feature-extraction-task',
   'name': 'Hugging Face Module'},
  'text2vec-openai': {'documentationHref': 'https://platform.openai.com/docs/guides/embeddings/what-are-embeddings',
   'name': 'OpenAI Module'}},
 'version': '1.23.0'}

## Step 2: Add data to Weaviate

### Add class definition

The equivalent of a SQL "table", or noSQL "collection" is called a "class" in Weaviate.

In case I created a demo class - let's delete it.

In [5]:
client.collections.delete("Question")

And create a new class definition here.
We'll set up a class called "Question" with:
- A "vectorizer" -> which will convert data to vectors, which represent meaning,
- A "generative" module -> which will allow us to use LLMs with our data, and
- Properties to save our quiz data (which are like SQL columns).
    - Just the question and answer for now

In [6]:
client.collections.create(
    name="Question",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(),
    generative_config=wvc.config.Configure.Generative.openai(),
    properties=[
        wvc.config.Property(
            name="Question",
            data_type=wvc.DataType.TEXT,
        ),
        wvc.config.Property(
            name="Answer",
            data_type=wvc.DataType.TEXT,
        )
    ]
)

{"level":"info","msg":"Created shard question_UzLPXGzS2W3v in 1.456458ms","time":"2024-01-10T13:39:39Z"}
{"action":"hnsw_vector_cache_prefill","count":1000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-10T13:39:39Z","took":49584}


> Tip: You can get example class definitions in our documentation:
> - https://weaviate.io/developers/weaviate/manage-data/classes#example-class-configurations

Was our class created successfully? Let's take a look

In [7]:
client.collections.list_all()

{'WikiChunks_sections': _CollectionConfigSimple(name='WikiChunks_sections', description=None, generative_config=_GenerativeConfig(generator=<GenerativeSearches.OPENAI: 'generative-openai'>, model={}), properties=[_Property(description='Article title', index_filterable=True, index_searchable=True, name='title', tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', data_type=<DataType.TEXT: 'text'>, nested_properties=None), _Property(description='Text chunk', index_filterable=True, index_searchable=True, name='chunk', tokenization=<Tokenization.WORD: 'word'>, vectorizer_config=_PropertyVectorizerConfig(skip=False, vectorize_property_name=True), vectorizer='text2vec-openai', data_type=<DataType.TEXT: 'text'>, nested_properties=None), _Property(description='Chunk number - 1 index', index_filterable=True, index_searchable=False, name='chunk_number', tokenization=None, vectorizer_config=_

### Add data

We'll add actual objects (SQL rows) to our data. 

First, let's build objects to add - and take a look at a couple.

In [8]:
for o in data[:2]:
    obj_body = {
        "question": o["Question"],
        "answer": o["Answer"],
    }
    print(obj_body)

{'question': 'Abraham Lincoln died across the street from this theatre on April 15, 1865', 'answer': "Ford's Theatre (the Ford Theatre accepted)"}
{'question': 'Any pigment on the wall so faded you can barely see it', 'answer': 'faint paint'}


> If it all looks fine - let's add objects:
> - https://weaviate.io/developers/weaviate/manage-data/import

In [9]:
from weaviate.util import generate_uuid5

question = client.collections.get("Question")

object_list = list()
for o in data:
    obj_body = {
        "question": o["Question"],
        "answer": o["Answer"],
    }
    wv_obj = wvc.data.DataObject(
        properties=obj_body,
        uuid=generate_uuid5(obj_body)
    )
    object_list.append(wv_obj)

response = question.data.insert_many(object_list)

{"level":"info","msg":"Completed loading shard wikichunk_XM7IgxRIHAIJ in 1.656333ms","time":"2024-01-10T13:39:40Z"}
{"action":"hnsw_vector_cache_prefill","count":3000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-10T13:39:40Z","took":66333}
{"level":"info","msg":"Completed loading shard wikichunks_sections_dXeMrdtLvCXu in 7.474292ms","time":"2024-01-10T13:39:40Z"}
{"level":"info","msg":"Completed loading shard wikichunk_section_ODyuLaNtSMRn in 7.521125ms","time":"2024-01-10T13:39:40Z"}
{"action":"hnsw_vector_cache_prefill","count":5000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-10T13:39:40Z","took":34312416}
{"action":"hnsw_vector_cache_prefill","count":5000,"index_id":"main","level":"info","limit":1000000000000,"msg":"prefilled vector cache","time":"2024-01-10T13:39:40Z","took":34577625}


In [10]:
print(response.has_errors)

False


In [11]:
print(response)

BatchObjectReturn(all_responses=[UUID('f1819cbc-5a70-52be-adf9-4afcff2c3195'), UUID('a8a00cb0-bee8-5bde-b7d1-245939c8778a'), UUID('29fc6b7f-71d7-5671-b86b-42062b09e042'), UUID('7bbf8fca-f31b-5a9c-b7fe-122d2d3b8f9a'), UUID('5bf5514d-a7a1-519f-aa05-2e8bb815d1f5'), UUID('cdd58b59-88d1-50e9-9efa-06002150a205'), UUID('8520461b-2d60-590a-938c-59bc65ff4d3c'), UUID('f06f25c4-f9bd-5843-8af2-6e4fb6d43516'), UUID('d174b6d2-609d-5e68-b6a8-2d520a9bcfda'), UUID('db5d5a31-6728-566f-90a3-6e3bc42113eb'), UUID('250914c3-a317-5fd1-accd-5888bf109fa2'), UUID('6139dff0-b18f-5188-a888-85ddc4851482'), UUID('06065ece-2e17-5a82-a8d4-ae4245957773'), UUID('c1167274-21ca-570a-9e55-cc4fab3c1513'), UUID('53986f4f-6283-5470-9571-1c8a27a034ad'), UUID('1d5ed28a-2bba-5d73-a8e5-324d3079265b'), UUID('ab841cc2-ecc2-50ab-aa67-e77d90f51601'), UUID('95bbf75c-7bc7-5b20-87af-8c91b9a1a94f'), UUID('dab9f9fa-8c4f-5d93-baae-e3ea4062867d'), UUID('bf64ea39-b55d-5962-8dd6-c51c09416e65'), UUID('df1dece0-d6b5-55fd-abe8-3c3be13f6e24'), U

#### Confirm data load

Do we have data? 

Let's get an object count

In [12]:
questions = client.collections.get("Question")

question.aggregate.over_all(total_count=True)

_AggregateReturn(properties={}, total_count=1000)

Does the data look right?

Let's grab a few objects from Weaviate!

In [13]:
response = questions.query.fetch_objects(limit=3)
for o in response.objects:
    print(o.properties)

{'answer': 'a stun gun', 'question': "An immobilizer used by the police (don't taze me, bro!)"}
{'answer': 'Brussels', 'question': 'NATO is headquartered on Blvd. Leopold III in this European capital'}
{'answer': 'Dutch', 'question': 'The standard form of this language developed from that spoken in Amsterdam & nearby cities'}


Let's pause for a second - because we've done a lot!

#### What did we just do?

Here is a conceptual diagram

![img](https://github.com/weaviate-tutorials/intro-workshop/blob/main/images/object_import_process_full.png?raw=1)

## Step 3: Work with the data

Let's try a few more involved queries

### Filtering (similar to WHERE filter in SQL)

Let's find objects that meet a particular condition.

In [14]:
response = questions.query.fetch_objects(
    filters=wvc.query.Filter("question").like("*history*"),
    limit=3
    )

for o in response.objects:
    print(o.properties["question"])

You're in the Army now--in 1940 FDR instituted the first peacetime one of these in U.S. history
A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles
The Drake Well Museum in Titusville, Penn. is dedicated to the history of this industry


But this does not rank the result in any meaningful way. 

For that, we need a keyword search (as opposed to a keyword *filter*).

### Keyword search

Unlike a keyword filter, a keyword search will search for, and rank results based on the frequency of the keyword.

In [15]:
response = questions.query.bm25(
    query="history",
    limit=10,
    return_metadata=wvc.query.MetadataQuery(score=True, last_update_time=True)
)

for o in response.objects:
    print(o.metadata.score)
    print(o.metadata.last_update_time)
    print(o.properties)

2.520803213119507
2024-01-10 13:39:47.673000+00:00
{'answer': '"A Brief History Of Time In A Bottle"', 'question': "Stephen Hawking's 1988 bio of the universe that was a No. 1 hit for Jim Croce"}
1.8396204710006714
2024-01-10 13:39:50.168000+00:00
{'answer': 'Oil', 'question': 'The Drake Well Museum in Titusville, Penn. is dedicated to the history of this industry'}
1.7712442874908447
2024-01-10 13:39:44.439000+00:00
{'answer': 'the Field Museum', 'question': 'What was once the Chicago Natural History Museum is now called this, after its founder'}
1.6486854553222656
2024-01-10 13:39:45.938000+00:00
{'answer': 'Greyhound', 'question': 'A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles'}
1.6486854553222656
2024-01-10 13:39:51.883000+00:00
{'answer': 'the draft', 'question': "You're in the Army now--in 1940 FDR instituted the first peacetime one of these in U.S. history"}


### Semantic search

A semantic search, on the other hand, searches objects based on similarity

In [16]:
response = questions.query.near_text(
    query="history",
    limit=3,
    return_metadata=wvc.query.MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.metadata)
    print(json.dumps(o.properties, indent=2))

_MetadataReturn(creation_time=None, last_update_time=None, distance=0.20320743322372437, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
{
  "answer": "Greyhound",
  "question": "A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles"
}
_MetadataReturn(creation_time=None, last_update_time=None, distance=0.20645064115524292, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
{
  "answer": "Shinto",
  "question": "Compiled in 712, the Kojiki, \"Records of Ancient Matters\", is one of this religion's oldest texts"
}
_MetadataReturn(creation_time=None, last_update_time=None, distance=0.210382342338562, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
{
  "answer": "\"A Brief History Of Time In A Bottle\"",
  "question": "Stephen Hawking's 1988 bio of the universe that was a No. 1 hit for Jim Croce"
}


#### How does this work?

- Under the hood, this uses a vector search. It looks for objects which are the most similar to a text input.
- We can inspect the similarity along with the results.

In [17]:
response = questions.query.near_text(
    query="history",
    limit=3,
    return_metadata=wvc.query.MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.metadata.distance)
    print(json.dumps(o.properties, indent=2))

0.20320743322372437
{
  "answer": "Greyhound",
  "question": "A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles"
}
0.20645064115524292
{
  "answer": "Shinto",
  "question": "Compiled in 712, the Kojiki, \"Records of Ancient Matters\", is one of this religion's oldest texts"
}
0.210382342338562
{
  "answer": "\"A Brief History Of Time In A Bottle\"",
  "question": "Stephen Hawking's 1988 bio of the universe that was a No. 1 hit for Jim Croce"
}


This is where "vectors" come in. 

Each object in Weaviate includes a vector - like so:

In [18]:
response = questions.query.near_text(
    query="history",
    limit=3,
    include_vector=True,
    return_metadata=wvc.query.MetadataQuery(distance=True)
)

for o in response.objects:
    print(o.metadata.distance)
    print(o.vector)
    print(json.dumps(o.properties, indent=2))

0.20320743322372437
[-0.015382598154246807, -0.010956727899610996, 0.002550272736698389, -0.03815963864326477, -0.026123428717255592, 0.01084877923130989, 0.006982889957726002, -0.014599974267184734, -0.013453026302158833, -0.02871418185532093, -0.02079349383711815, 0.015463558956980705, -0.0072392662987113, -0.020618079230189323, -0.028039507567882538, 0.010909500531852245, 0.014181675389409065, 0.014006259851157665, 0.013399052433669567, 0.006807474419474602, -0.004520324990153313, -0.006014730781316757, 0.010025675408542156, -0.009931220673024654, 0.006412789225578308, 0.004581045359373093, 0.027742650359869003, -0.01113888993859291, -0.0021100470330566168, 0.013102195225656033, -0.005383909214287996, -0.0023006428964436054, -0.016151728108525276, 0.0007484679808840156, 0.004955490585416555, -0.032060571014881134, 0.001207668916322291, 0.009141851216554642, 0.018621038645505905, 0.013243877328932285, -0.011644896119832993, 0.0032047079876065254, -0.003977211192250252, -0.00098165264

These vector representations come from deep learning models to those that power LLMs. They capture meaning, and are called vector "embeddings".

### Generative search

A generative search transforms your data at retrieval time. 

In [19]:
response = questions.generate.near_text(
    query="history",
    limit=5,
    single_prompt="Write a tweet about {question} as an interesting factoid.",
    grouped_task="Summarize this into bullet points"
)

print(response.generated)
for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))

- Greyhound: Founded in Hibbing, Minn. in 1914, a museum traces its history using Hupmobiles
- Shinto: The Kojiki, compiled in 712, is one of the oldest texts of this religion
- "A Brief History Of Time In A Bottle": Stephen Hawking's 1988 bio of the universe, a No. 1 hit for Jim Croce
- The Rijksmuseum: Dutch national art museum with origins in one founded by Louis Bonaparte in 1808
- John James Audubon: Artist and naturalist who began writing "The Ornithological Biography" in 1831
"Did you know? 🚌 A fascinating piece of history lies in Hibbing, Minn.! 🏞️ The local museum takes you back to 1914, where a bus company was founded using Hupmobiles! 🚍 Explore the rich heritage and evolution of transportation at this hidden gem. 🌟 #Hibbing #Museum #TransportationHistory"
{
  "answer": "Greyhound",
  "question": "A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles"
}
"📜 Fun fact: The Kojiki, also known as 'Records of Ancient Matters', was com

You can see here ⬆️ that each object has been transformed into a tweet by the LLM based on our prompt.

You can ask LLMs to perform all sorts of tasks

In [20]:
response = questions.generate.near_text(
    query="history",
    limit=3,
    single_prompt="Translate {question} into French."
)

for o in response.objects:
    print(o.generated)
    print(json.dumps(o.properties, indent=2))

Un musée à Hibbing, dans le Minnesota, retrace l'histoire de cette compagnie de bus fondée en 1914 en utilisant des Hupmobiles.
{
  "answer": "Greyhound",
  "question": "A Hibbing, Minn. museum traces the history of this bus company founded there in 1914 using Hupmobiles"
}
Traduit en 712, le Kojiki, "Chroniques des temps anciens", est l'un des plus anciens textes de cette religion.
{
  "answer": "Shinto",
  "question": "Compiled in 712, the Kojiki, \"Records of Ancient Matters\", is one of this religion's oldest texts"
}
Traduisez la biographie de l'univers de Stephen Hawking de 1988, qui a été un succès numéro 1 pour Jim Croce, en français.
{
  "answer": "\"A Brief History Of Time In A Bottle\"",
  "question": "Stephen Hawking's 1988 bio of the universe that was a No. 1 hit for Jim Croce"
}


The LLM is multi-lingual!

You can also send groups of results to the LLM with Weaviate.

In [21]:
response = questions.generate.near_text(
    query="history",
    limit=3,
    grouped_task="Write a poem about these facts"
)

print(response.generated)
for o in response.objects:
    print(json.dumps(o.properties, indent=2))

In the land of Hibbing, where history resides,
A museum stands tall, where knowledge abides.
Tracing the footsteps of a company grand,
Founded in 1914, by a visionary hand.

Greyhound, the name that echoes through time,
A bus company born, with a purpose sublime.
Using Hupmobiles, they paved the way,
Connecting people, day after day.

In ancient Japan, where traditions unfold,
A religion emerged, with stories untold.
Shinto, the faith that holds sacred ground,
With Kojiki, its ancient text renowned.

Compiled in 712, a treasure so old,
The Records of Ancient Matters, we are told.
A glimpse into a world of gods and lore,
A testament to beliefs that forever endure.

And in the realm of science, where wonders abound,
Stephen Hawking's brilliance, so profound.
A Brief History Of Time In A Bottle, he wrote,
Unraveling the universe, note by note.

A No. 1 hit, not in music alone,
But in the minds of those who sought to be known.
Jim Croce's voice, carrying Hawking's tale,
A harmony of knowle

## What more can we do with Weaviate?

Here is a demo instance that you can connect to and try out. 

Like many of our production clusters, we have a read-only API key set up that you can use.

This instance is populated with Wiki articles.

In [22]:
client = weaviate.connect_to_wcs(
    cluster_url="https://hha2nvjsruetknc5vxwrwa.c0.europe-west2.gcp.weaviate.cloud",
    auth_credentials=weaviate.AuthApiKey("nMZuw1z1zVtnjkXXOMGx9Ows7YWGsakItdus"),  # ReadOnly key
    headers={
        "X-OpenAI-Api-Key": os.getenv("OPENAI_APIKEY")
    }
)

wiki_chunks = client.collections.get("WikiChunk")
response = wiki_chunks.query.near_text(query="American revolution", limit=2)

for o in response.objects:
    print(o)

_Object(uuid=_WeaviateUUIDInt('283cdc5d-84a2-55de-b186-8f00106bc877'), metadata=_MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'chunk_number': 116, 'title': 'Late modern period', 'chunk': '*: North America | ined against the British to defend that self-governance in the armed conflict from 1775 to 1783 known as the American Revolutionary War (also called American War of Independence).\nThe American Revolut'}, references=None, vector=None)
_Object(uuid=_WeaviateUUIDInt('e1b8ac81-7cd1-5a3b-a6d5-a99f88c66fce'), metadata=_MetadataReturn(creation_time=None, last_update_time=None, distance=None, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'chunk_number': 117, 'title': 'Late modern period', 'chunk': '*: North America | The American Revolution began with fighting at Lexington and Concord. On July 4, 1776, they iss

Using Weaviate, we can talk to the collection of articles!

In [23]:
wiki_chunks = client.collections.get("WikiChunk")

resp = wiki_chunks.generate.near_text(
    query="space travel",
    limit=4,
    grouped_task="What significant events are mentioned here? Summarize into bullet points",
    single_prompt="Translate {title} to a random Asian or European language. Your choice."
)

print("========== Grouped task generation ==========")
print(resp.generated)
print("========== Single prompt generation ==========")
for o in resp.objects:
    print(o.generated, o.properties["title"])

========== Grouped task generation ==========
- Humanity's perspective: Voyager 1 spacecraft is the furthest object made by humankind and the first in interstellar space. Probes have returned samples from comets and asteroids.
- European decline and the 20th century: Space Age brought the first human spaceflight during the Vostok program and reached its peak with the Apollo program, including the 1969 moon landing.
- Modes: Suborbital spaceflight is the fastest transport system. Probes have been sent to all the planets of the Solar System.
- Other modes: Spacecraft technology is used to put satellites into orbit and conduct scientific experiments. Man has also landed on the moon.
========== Single prompt generation ==========
Japanese: 太陽系 (Taiyōkei) Solar System
Late modern period translated to Japanese: 近代後期 (Kindai kōki) Late modern period
I will translate "Transport" to Japanese.

Transport: トランスポート (Toransupōto) Transport
I will translate "Transport" to Japanese.

Transport: トランスポ

And a lot more. 

Weaviate makes it easy for you to work with your data and these AI models, at scale. As a vector database, we deal with data stores with 10s or 100s of M objects!